## Формулируем цели

 Нам предстоит проверить следущие предположения:

   1. Зависит ли вероятность оплаты от выбранного пользователем уровня сложности бесплатных тренировок?
   2. Существует ли разница во времени между пользователями с разным уровнем сложности и их первой оплатой?
   3. Дополнительно можно проверить, существует ли разница во времени между событиями регистрации и оплаты.

 Проверку будем проводить на основе данных пользователей, которые зарегистрировались в 2017 году (с 1 января по 31 декабря 2017 года включительно).

### 1. Импортируем библиотеки и получим данные из базы данных

In [1]:
# Импортируем библиотеки, которые нам потребуются
import pandas as pd

In [2]:
# Наши данные хранятся в файлах events.csv и purchase.csv которые были предварительно выгружены из БД.
# Загрузим файл events.csv, содержащий данные по событиям пользователей в датафрейм events_df
events_df = pd.read_csv('events.csv')
# А файл purchase.csv, содержащий данные по оплатам пользователей - в датафрейм purchase_df
purchase_df = pd.read_csv('purchase.csv')

### 2. Исследуем датафреймы.

In [3]:
# Выведем первые 5 строк фрейма, содержащего данные о действиях пользователей.    
events_df.head(5)

,event_type,selected_level,start_time,tutorial_id,user_id,id
0,registration,NaN,2017-01-01 03:48:40,NaN,27832,80308
1,registration,NaN,2017-01-01 04:07:25,NaN,27833,80309
2,registration,NaN,2017-01-01 08:35:10,NaN,27834,80310
3,registration,NaN,2017-01-01 11:54:47,NaN,27835,80311
4,registration,NaN,2017-01-01 13:28:07,NaN,27836,80312


In [4]:
# Посмотрим на наши данные.  Как мы видим датафрейм состоит из 6 столбцов в котором 66929 строк
# В столбцах selected_level и tutorial_id содержатся пропущенные значения. Это связано с тем что не все из указанных 
# параметров обязательно присутствуют в каждом событии. 
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66929 entries, 0 to 66928
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   event_type      66929 non-null  object 
 1   selected_level  8338 non-null   object 
 2   start_time      66929 non-null  object 
 3   tutorial_id     32942 non-null  float64
 4   user_id         66929 non-null  int64  
 5   id              66929 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 3.1+ MB


In [5]:
# Преобразуем столбец "start_time" в тип datetime с помощью метода pd.to_datetime():
events_df['start_time'] = pd.to_datetime(events_df['start_time'])
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66929 entries, 0 to 66928
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_type      66929 non-null  object        
 1   selected_level  8338 non-null   object        
 2   start_time      66929 non-null  datetime64[ns]
 3   tutorial_id     32942 non-null  float64       
 4   user_id         66929 non-null  int64         
 5   id              66929 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 3.1+ MB


In [6]:
# Сделаем срез датафрейма по событию 'level_choice'. Как видно, теперь пропущенные значения содержатся только в столбце tutorial_id.
# Это связано с тем, что для события 'level_choice' не предусмотрена запись параметра 'tutorial_id'
events_df[events_df['event_type'] == 'level_choice'].info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8338 entries, 19 to 66927
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_type      8338 non-null   object        
 1   selected_level  8338 non-null   object        
 2   start_time      8338 non-null   datetime64[ns]
 3   tutorial_id     0 non-null      float64       
 4   user_id         8338 non-null   int64         
 5   id              8338 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 456.0+ KB


In [7]:
# Для оценки значений в столбцах используем метод describe. 
# Запустим его с параметром include='all' чтобы вывести информацию по всем столбцам.
events_df.describe(include='all')

,event_type,selected_level,start_time,tutorial_id,user_id,id
count,66929,8338,66929,32942.000000,66929.000000,66929.000000
unique,5,3,66779,NaN,NaN,NaN
top,registration,medium,2017-03-13 09:37:43,NaN,NaN,NaN
freq,19914,4642,9,NaN,NaN,NaN
first,NaN,NaN,2017-01-01 03:48:40,NaN,NaN,NaN
last,NaN,NaN,2017-12-31 10:05:18,NaN,NaN,NaN
mean,NaN,NaN,NaN,40529.649323,37777.074273,113772.001016
std,NaN,NaN,NaN,5211.593585,5748.910912,19320.884180
min,NaN,NaN,NaN,31505.000000,27832.000000,80308.000000
25%,NaN,NaN,NaN,36007.000000,32847.000000,97040.000000


In [8]:
# Посмотрим какие уникальные значения содержатся в столбце event_type. Это пригодится в дальнейшем для формирования нужных датафреймов
events_df['event_type'].unique()

array(['registration', 'tutorial_start', 'tutorial_finish',
       'level_choice', 'training_choice'], dtype=object)

In [9]:
# Также посмотрим какие уникальные значения имеются в столбце выбора тренировок
events_df['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

In [10]:
# Узнаем, какое количество уникальных пользователей в нашем датасете
events_df['user_id'].nunique()

19914

In [11]:
# Выведем первые 5 строк фрейма, содержащего данные об оплатах пользователей.  
purchase_df.head(5)

,user_id,start_time,amount,id
0,37878,2017-06-30 17:05:21,150,17668
1,47216,2017-12-22 06:30:31,25,18396
2,35532,2017-05-21 04:23:32,150,17475
3,42583,2017-10-17 13:52:14,100,18027
4,32321,2017-03-20 06:51:27,150,17202


In [12]:
# Исследуем датафрейм на наличие пропусков и типов данных в столбцах.
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     1600 non-null   int64 
 1   start_time  1600 non-null   object
 2   amount      1600 non-null   int64 
 3   id          1600 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 50.1+ KB


In [13]:
# Преобразуем столбец "start_time" в тип datetime с помощью метода pd.to_datetime():
purchase_df['start_time'] = pd.to_datetime(purchase_df['start_time'])
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     1600 non-null   int64         
 1   start_time  1600 non-null   datetime64[ns]
 2   amount      1600 non-null   int64         
 3   id          1600 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 50.1 KB


In [14]:
# Оценим параметры датасета при помощи метода describe.
purchase_df.describe(include='all')

,user_id,start_time,amount,id
count,1600.000000,1600,1600.000000,1600.000000
unique,NaN,1600,NaN,NaN
top,NaN,2017-02-15 16:17:28,NaN,NaN
freq,NaN,1,NaN,NaN
first,NaN,2017-01-03 18:53:43,NaN,NaN
last,NaN,2018-01-06 07:14:25,NaN,NaN
mean,37752.766250,NaN,110.734375,17645.505625
std,5822.621784,NaN,54.696628,462.038637
min,27845.000000,NaN,25.000000,16845.000000
25%,32815.750000,NaN,50.000000,17245.750000


###  3. Выделим группы пользователей

In [15]:
# Выделим группу пользователей, которые выбрали уровень сложности "easy" и определим их количество. 
# Запишем группу пользователей в переменную "easy_level_users", а в переменную "easy_level_users_count" - их количество.

print(events_df[events_df['selected_level'] == 'easy']['user_id'].nunique())
easy_level_users = events_df[events_df['selected_level'] == 'easy']['user_id']
easy_level_users_count = events_df[events_df['selected_level'] == 'easy']['user_id'].nunique()

2447


In [16]:
# Выделим группу пользователей, которые выбрали уровень сложности "medium" и определим их количество. 
# Запишем группу пользователей в переменную "medium_level_users", а в переменную "medium_level_users_count" - их количество.

print(events_df[events_df['selected_level'] == 'medium']['user_id'].nunique())
medium_level_users = events_df[events_df['selected_level'] == 'medium']['user_id']
medium_level_users_count = events_df[events_df['selected_level'] == 'medium']['user_id'].nunique()

4642


In [17]:
# Выделим группу пользователей, которые выбрали уровень сложности "hard" и определим их количество. 
# Запишем группу пользователей в переменную "hard_level_users", а в переменную "hard_level_users_count" - их количество.

print(events_df[events_df['selected_level'] == 'hard']['user_id'].nunique())
hard_level_users = events_df[events_df['selected_level'] == 'hard']['user_id']
hard_level_users_count = events_df[events_df['selected_level'] == 'hard']['user_id'].nunique()

1249


In [18]:
# Выделим группу пользователей, которые оплатили тренировки и выбрали уровень сложности "easy" 
# Запишем группу пользователей в отдельный датафрейм "purchase_df_easy", а в переменную "purchase_easy_users_count" - их количество.

purchase_df_easy = purchase_df[purchase_df['user_id'].isin(easy_level_users)]
print(purchase_df_easy.user_id.nunique())
purchase_easy_users_count = purchase_df_easy.user_id.nunique()

189


In [19]:
# Выделим группу пользователей, которые оплатили тренировки и выбрали уровень сложности "medium" 
# Запишем группу пользователей в отдельный датафрейм "purchase_df_medium", а в переменную "purchase_medium_users_count" - их количество.

purchase_df_medium = purchase_df[purchase_df['user_id'].isin(medium_level_users)]
print(purchase_df_medium.user_id.nunique())
purchase_medium_users_count = purchase_df_medium.user_id.nunique()

969


In [20]:
# Выделим группу пользователей, которые оплатили тренировки и выбрали уровень сложности "hard" 
# Запишем группу пользователей в отдельный датафрейм "purchase_df_hard", а в переменную "purchase_hard_users_count" - их количество.

purchase_df_hard = purchase_df[purchase_df['user_id'].isin(hard_level_users)]
print(purchase_df_hard.user_id.nunique())
purchase_hard_users_count = purchase_df_hard.user_id.nunique()

442


### 4. Рассчитаем для каждой группы пользователей процент оплат

In [21]:
# Оценим процент пользователей, выбравших уровень сложности "easy" от числа пользователей, которые купили тренировки. 
# Запишем результат в переменную percent_easy_level:

percent_easy_level = purchase_easy_users_count / easy_level_users_count
print('Процент пользователей, которые оплатили тренировки (от числа пользователей, который выбрали уровень тренировок "easy"): {:.2%}'.format(percent_easy_level))

Процент пользователей, которые оплатили тренировки (от числа пользователей, который выбрали уровень тренировок "easy"): 7.72%


In [22]:
# Оценим процент пользователей, выбравших уровень сложности "medium" от числа пользователей, которые купили тренировки. 
# Запишем результат в переменную percent_medium_level:

percent_medium_level = purchase_medium_users_count / medium_level_users_count
print('Процент пользователей, которые оплатили тренировки (от числа пользователей, который выбрали уровень тренировок "medium"): {:.2%}'.format(percent_medium_level))

Процент пользователей, которые оплатили тренировки (от числа пользователей, который выбрали уровень тренировок "medium"): 20.87%


In [23]:
# Оценим процент пользователей, выбравших уровень сложности "hard" от числа пользователей, которые купили тренировки. 
# Запишем результат в переменную percent_hard_level:

percent_hard_level = purchase_hard_users_count / hard_level_users_count
print('Процент пользователей, которые оплатили тренировки (от числа пользователей, который выбрали уровень тренировок "hard"):{:.2%}'.format(percent_hard_level))

Процент пользователей, которые оплатили тренировки (от числа пользователей, который выбрали уровень тренировок "hard"):35.39%


### 5. Создадим объединенный датафрейм

In [24]:
# Добавим в датафрейм purchase_df столбец "event_type" со значением "purchase"
purchase_df['event_type'] = 'purchase'
purchase_df

,user_id,start_time,amount,id,event_type
0,37878,2017-06-30 17:05:21,150,17668,purchase
1,47216,2017-12-22 06:30:31,25,18396,purchase
2,35532,2017-05-21 04:23:32,150,17475,purchase
3,42583,2017-10-17 13:52:14,100,18027,purchase
4,32321,2017-03-20 06:51:27,150,17202,purchase
...,...,...,...,...,...
1595,34166,2017-04-20 13:04:21,50,17358,purchase
1596,36640,2017-06-07 08:26:28,150,17556,purchase
1597,42221,2017-10-07 11:14:19,100,17990,purchase
1598,37754,2017-06-25 11:34:08,50,17637,purchase


In [25]:
# Переименуем столбец 'id' в датафрейме "purchase_df" и столбец 'id' в датафрейме "events_df".
purchase_df = purchase_df.rename(columns={'id':'purchase_id'})
events_df = events_df.rename(columns={'id':'event_id'})

In [26]:
# Теперь мы можем объединить два датафрейма в один для того чтобы можно было создать датафреймы с необходимыми нам данными.
total_df = pd.concat([events_df,purchase_df],sort=False)

In [27]:
# Сбросим индексы в новом датафрейме (они дублировались после объединения поэтому бесполезны для нас), 
# а также отсортируем датафрейм по возрастанию по столбцу 'start_time'.
total_df = total_df.reset_index(drop=True).sort_values('start_time')

In [28]:
# Выведем первые 10 строк объединенного датафрейма.
total_df.head(10)

,event_type,selected_level,start_time,tutorial_id,user_id,event_id,amount,purchase_id
0,registration,NaN,2017-01-01 03:48:40,NaN,27832,80308.0,NaN,NaN
1,registration,NaN,2017-01-01 04:07:25,NaN,27833,80309.0,NaN,NaN
2,registration,NaN,2017-01-01 08:35:10,NaN,27834,80310.0,NaN,NaN
3,registration,NaN,2017-01-01 11:54:47,NaN,27835,80311.0,NaN,NaN
4,registration,NaN,2017-01-01 13:28:07,NaN,27836,80312.0,NaN,NaN
5,registration,NaN,2017-01-01 14:08:40,NaN,27837,80313.0,NaN,NaN
6,registration,NaN,2017-01-01 14:42:58,NaN,27838,80314.0,NaN,NaN
7,tutorial_start,NaN,2017-01-01 14:54:40,31505.0,27836,80315.0,NaN,NaN
8,tutorial_start,NaN,2017-01-01 15:00:51,31506.0,27835,80316.0,NaN,NaN
9,tutorial_finish,NaN,2017-01-01 15:06:15,31506.0,27835,80317.0,NaN,NaN


In [29]:
# Отобразим информацию о строках и столбцах
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68529 entries, 0 to 68460
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_type      68529 non-null  object        
 1   selected_level  8338 non-null   object        
 2   start_time      68529 non-null  datetime64[ns]
 3   tutorial_id     32942 non-null  float64       
 4   user_id         68529 non-null  int64         
 5   event_id        66929 non-null  float64       
 6   amount          1600 non-null   float64       
 7   purchase_id     1600 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 4.7+ MB


#### Для каждой группы создадим датафреймы

In [30]:
# Создадим датафрейм "easy_level_choice_df", в котором будут только пользователи, выбравшие уровень сложности "easy". 
# Оставим в новом датафрейме только необходимые нам столбцы 'user_id', 'start_time', а также переименуем 
# столбец 'start_time' в столбец 'easy_choice_time'. 
# В заключении проверим, что на каждого уникального пользователя приходится только одно событие выбора уровня сложности. 

easy_level_choice_df = total_df[total_df['selected_level'] == 'easy']
easy_level_choice_df = easy_level_choice_df[['user_id', 'start_time']].rename(columns={'start_time':'easy_choice_time'})
easy_level_choice_df['user_id'].value_counts().mean()

1.0

In [31]:
# Подкорректируем ранее созданный датафрейм "purchase_df_easy": оставим в датафрейме только необходимые нам 
# столбцы 'user_id', 'start_time', а также переименуем столбец 'start_time' в столбец 'easy_pay_time'.
# В заключении проверим, что на каждого уникального пользователя приходится только одно событие оплаты.

purchase_df_easy = purchase_df_easy[['user_id','start_time']].rename(columns={'start_time':'easy_pay_time'})
purchase_df_easy['user_id'].value_counts().mean()

1.0

In [32]:
# Создадим датафрейм "medium_level_choice_df", в котором будут только пользователи, выбравшие уровень сложности "medium". 
# Оставим в новом датафрейме только необходимые нам столбцы 'user_id', 'start_time', а также переименуем 
# столбец 'start_time' в столбец 'medium_choice_time'. 
# В заключении проверим, что на каждого уникального пользователя приходится только одно событие выбора уровня сложности. 

medium_level_choice_df = total_df[total_df['selected_level'] == 'medium']
medium_level_choice_df = medium_level_choice_df[['user_id','start_time']].rename(columns={'start_time':'medium_choice_time'})
medium_level_choice_df['user_id'].value_counts().mean()

1.0

In [33]:
# Подкорректируем ранее созданный датафрейм "purchase_df_medium": оставим в датафрейме только необходимые нам 
# столбцы 'user_id', 'start_time', а также переименуем столбец 'start_time' в столбец 'medium_pay_time'.
# В заключении проверим, что на каждого уникального пользователя приходится только одно событие оплаты.

purchase_df_medium = purchase_df_medium[['user_id','start_time']].rename(columns={'start_time':'medium_pay_time'})
purchase_df_medium['user_id'].value_counts().mean()

1.0

In [34]:
# Создадим датафрейм "hard_level_choice_df", в котором будут только пользователи, выбравшие уровень сложности "hard". 
# Оставим в новом датафрейме только необходимые нам столбцы 'user_id', 'start_time', а также переименуем 
# столбец 'start_time' в столбец 'hard_choice_time'. 
# В заключении проверим, что на каждого уникального пользователя приходится только одно событие выбора уровня сложности. 

hard_level_choice_df = total_df[total_df['selected_level'] == 'hard']
hard_level_choice_df = hard_level_choice_df[['user_id','start_time']].rename(columns={'start_time':'hard_choice_time'})
hard_level_choice_df['user_id'].value_counts().mean()

1.0

In [35]:
# Подкорректируем ранее созданный датафрейм "purchase_df_hard": оставим в датафрейме только необходимые нам 
# столбцы 'user_id', 'start_time', а также переименуем столбец 'start_time' в столбец 'hard_pay_time'.
# В заключении проверим, что на каждого уникального пользователя приходится только одно событие оплаты.

purchase_df_hard = purchase_df_hard[['user_id','start_time']].rename(columns={'start_time':'hard_pay_time'})
purchase_df_hard['user_id'].value_counts().mean()

1.0

#### Объединим датафреймы и посчитаем разницу во времени между событиями

In [36]:
# Объединим датафрейм с пользователями, которые выбрали уровень сложности "easy" с датафреймом содержащим информацию об оплатах.
easy_level_pay_users_df = easy_level_choice_df.merge(purchase_df_easy, on='user_id' ,how ='inner')
easy_level_pay_users_df.head(5)

,user_id,easy_choice_time,easy_pay_time
0,27884,2017-01-04 16:18:39,2017-01-08 19:37:34
1,28090,2017-01-09 21:34:23,2017-01-15 23:42:55
2,28182,2017-01-11 18:44:45,2017-01-12 02:46:01
3,28207,2017-01-11 21:10:51,2017-01-12 21:00:24
4,28254,2017-01-12 16:48:24,2017-01-19 22:08:40


In [37]:
# Создадим в датафрейме новый столбец ('timedelta') в который запишем разницу во времени между первой оплатой 
# и выбором уровня сложности. После этого посчитаем среднее значение времени.
easy_level_pay_users_df['timedelta'] = easy_level_pay_users_df['easy_pay_time'] - easy_level_pay_users_df['easy_choice_time']
print(easy_level_pay_users_df['timedelta'].mean())

3 days 14:58:52.941798


In [38]:
# Объединим датафрейм с пользователями, которые выбрали уровень сложности "medium" с датафреймом содержащим информацию об оплатах.
medium_level_pay_users_df = medium_level_choice_df.merge(purchase_df_medium, on='user_id', how='inner')
medium_level_pay_users_df.head(5)

,user_id,medium_choice_time,medium_pay_time
0,27973,2017-01-07 05:29:30,2017-01-13 21:50:00
1,27981,2017-01-07 10:46:14,2017-01-07 23:20:25
2,28010,2017-01-08 00:00:52,2017-01-10 05:32:47
3,28020,2017-01-08 14:47:35,2017-01-11 21:43:03
4,28033,2017-01-08 17:06:39,2017-01-16 05:08:41


In [39]:
# Создадим в датафрейме новый столбец ('timedelta') в который запишем разницу во времени между первой оплатой 
# и выбором уровня сложности. После этого посчитаем среднее значение времени.
medium_level_pay_users_df['timedelta'] = medium_level_pay_users_df['medium_pay_time'] - medium_level_pay_users_df['medium_choice_time']
print(medium_level_pay_users_df['timedelta'].mean())

3 days 23:14:13.165118


In [40]:
# Объединим датафрейм с пользователями, которые выбрали уровень сложности "hard" с датафреймом содержащим информацию об оплатах.
hard_level_pay_users_df = hard_level_choice_df.merge(purchase_df_hard, on='user_id', how='inner')
hard_level_pay_users_df.head(5)

,user_id,hard_choice_time,hard_pay_time
0,27845,2017-01-02 06:19:18,2017-01-03 18:53:43
1,27865,2017-01-04 05:56:32,2017-01-04 14:46:10
2,27910,2017-01-05 11:59:50,2017-01-07 12:11:34
3,27911,2017-01-05 17:39:02,2017-01-07 08:19:12
4,27940,2017-01-06 00:32:47,2017-01-07 13:16:41


In [41]:
# Создадим в датафрейме новый столбец ('timedelta') в который запишем разницу во времени между первой оплатой 
# и выбором уровня сложности. После этого посчитаем среднее значение времени.
hard_level_pay_users_df['timedelta'] = hard_level_pay_users_df['hard_pay_time'] - hard_level_pay_users_df['hard_choice_time']
print(hard_level_pay_users_df['timedelta'].mean())

3 days 07:20:41.420814


#### Дополнительное задание. Проверим, существует ли разница во времени между событиями регистрации и оплаты.

In [42]:
# Создадим датафрейм содержащий только события 'event_type' = 'registration'.  
registration_df = total_df[total_df['event_type'] == 'registration']
registration_df = registration_df[['user_id', 'start_time']].rename(columns={'start_time':'reg_time'})

In [43]:
# Создадим новый отдельный датафрейм содержащий только события 'event_type' = 'purchase'
purchase_df_1 = purchase_df[['user_id','start_time']].rename(columns={'start_time':'purchase_time'})

In [44]:
# Теперь объединим оба датафрейма и посчитаем разницу во времени между событиями регистрации и оплаты.
merge_df =registration_df.merge(purchase_df_1, on='user_id', how='inner')
merge_df.head(10)
merge_df['timedelta'] = merge_df['purchase_time'] - merge_df['reg_time']
print(merge_df['timedelta'].mean())

4 days 01:01:56.595000


### Выводы:

   
    1. Зависит ли вероятность оплаты от выбранного пользователем уровня сложности бесплатных тренировок?
    По результатам проведенного анализа выявлено что только около 8% пользователей, выбравших уровень сложности "easy" совершают оплату. Если посмотреть пользователей, выбравших уровень сложности "medium", то их уже около 21%. А среди пользователей, которые выбрали уровень сложности "hard" 35 % совершили оплату.
    Таким образом, мы видим, что с изменением уровня сложности тренировок от более легкого к более трудному, вероятность того что пользователей совершит оплату увеличивается.

    2. Существует ли разница во времени между пользователями с разным уровнем сложности и их первой оплатой?
        - Среди пользователей, выбравших уровень сложности "easy" среднее время между выбором уровня сложности и первой оплаты составляет в среднем 3 дня и 15 часов;
        - Среди пользователей, выбравших уровень сложности "medium" среднее время между выбором уровня сложности и первой оплаты составляет в среднем 3 дня и 23 часа;
        - Среди пользователей, выбравших уровень сложности "hard" среднее время между выбором уровня сложности и первой оплаты составляет в среднем 3 дня и 7,5 часов;
    Таким образом мы видим, что быстрее всего начинают платить пользователи, выбравшие уровень сложности "hard", им требуется в среднем 3 дня и 7,5 часов. Пользователям, выбравшим уровень сложности "easy" требуется в среднем на 7,5 часов больше чем пользователям с уровнем "hard". А пользователи, выбравшие уровень "medium", совершают первый платеж в среднем через 4 дня после выбора уровня сложности.
    
    3. Cуществует ли разница во времени между событиями регистрации и оплаты.
    Как показывает наш анализ, в среднем между событиями регистрации и оплаты проходит 4 дня и 1 час.